In [ ]:
import theano
theano.config.device = 'gpu'
theano.config.floatX = 'float32'

In [5]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    print(e)


In [6]:

from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch


C:\Users\man\AppData\Local\Temp\ipykernel_19400\3510533452.py:3: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [7]:
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

11490434/11490434 [==============================] - 4s 0us/step


In [8]:
def build_model(hp):  
    model = keras.Sequential()
    model.add(layers.Conv2D(filters=hp.Int('conv_1_filter', min_value=16, max_value=64, step=16),
                             kernel_size=hp.Choice('conv_1_kernel', values=[3,5]),
                             activation='relu', input_shape=(28,28,1)))
    model.add(layers.Conv2D(filters=hp.Int('conv_2_filter', min_value=16, max_value=64, step=16),
                             kernel_size=hp.Choice('conv_2_kernel', values=[3,5]),
                             activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(units=hp.Int('dense_1_units', min_value=32, max_value=128, step=32),
                           activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))

    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2, 1e-3, 1e-4])),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model


In [10]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=1,
    directory='my_dir',
    project_name='helloworld'
   
)

tuner.search_space_summary()


INFO:tensorflow:Reloading Tuner from my_dir\helloworld\tuner0.json
Search space summary
Default search space size: 6
conv_1_filter (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': 'linear'}
conv_1_kernel (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5], 'ordered': True}
conv_2_filter (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': 'linear'}
conv_2_kernel (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5], 'ordered': True}
dense_1_units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [11]:
tuner.search(train_images, train_labels, epochs=5, validation_data=(test_images, test_labels))

best_model = tuner.get_best_models()[0]
best_hyperparameters = tuner.get_best_hyperparameters()[0]

print(best_hyperparameters.values)
test_loss, test_acc = best_model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)


Trial 5 Complete [00h 05m 27s]
val_accuracy: 0.98089998960495

Best val_accuracy So Far: 0.9876999855041504
Total elapsed time: 00h 18m 40s
INFO:tensorflow:Oracle triggered exit


{'conv_1_filter': 48, 'conv_1_kernel': 5, 'conv_2_filter': 32, 'conv_2_kernel': 5, 'dense_1_units': 128, 'learning_rate': 0.0001}
313/313 [==============================] - 2s 6ms/step - loss: 0.0436 - accuracy: 0.9877
Test accuracy: 0.9876999855041504


Trial 5 Complete [00h 05m 27s]
val_accuracy: 0.98089998960495

Best val_accuracy So Far: 0.9876999855041504
Total elapsed time: 00h 18m 40s
INFO:tensorflow:Oracle triggered exit
{'conv_1_filter': 48, 'conv_1_kernel': 5, 'conv_2_filter': 32, 'conv_2_kernel': 5, 'dense_1_units': 128, 'learning_rate': 0.0001}
313/313 [==============================] - 2s 6ms/step - loss: 0.0436 - accuracy: 0.9877
Test accuracy: 0.9876999855041504